# Taking the derivative by FFT
Suppose $g(x)$ is an absolutely continuous differentiable function, and both $g$ and its derivative $g′$ are integrable. Then, the Fourier transform of the derivative is given by (__[Wikipedia](https://en.wikipedia.org/wiki/Fourier_transform#Differentiation)__):
\begin{equation}
\mathcal{F}\left\{ \frac{d}{dx} g(x)\right\} = i 2\pi \xi \ \hat{g}(\xi).
\end{equation}
Note that the differential operator transformed into a multiplication.

This in principle allows to compute the derivative of any function $g(x)$ by first computing its Fourier transformation and then inversely transforming the product: 
\begin{equation}
\frac{d^n}{d^n x} g(x) = \mathcal{F^{-1}}{ \left( \mathcal{F}{ \left( \frac{d^n}{d^n x} g(x) \right)} \right)}  = \mathcal{F^{-1}}{ \left( (i 2\pi \xi)^n \ \hat{g}(\xi) \right)}.
\end{equation}

For analytic functions this is often only of limited helpfulness since you need to solve two integrals.
However, for a numerically sampled function we can just use the FFT.

<b>Tip:</b> It should be noted, that this is only feasable to use numerically when $g(x) \rightarrow 0$ as $x$ approaches the boundary of the coordinate grid.

## Implementation

In [ ]:
import numpy as np
from bokeh.io import output_notebook
output_notebook()

# test function in position space
f1_x = lambda x, a: np.cos(x)/np.exp((-a + x)**2/25.)
# first derivative of the test function
f1_x_d1 = lambda x, a: (-2*(-a + x)*np.cos(x))/(25.*np.exp((-a + x)**2/25.)) - np.sin(x)/np.exp((-a + x)**2/25.)
# second derivative of the test function
f1_x_d2 = lambda x, a: -(np.cos(x)/np.exp((-a + x)**2/25.)) + (-2/(25.*np.exp((-a + x)**2/25.)) + (4*(-a + x)**2)/(625.*np.exp((-a + x)**2/25.)))*np.cos(x) + (4*(-a + x)*np.sin(x))/(25.*np.exp((-a + x)**2/25.))
# test function in frequency space
f1_k = lambda k, a: (5*np.exp(1.j*a*(-1 + k) - (25*(1 + k)**2)/4.)*(np.exp(2*1.j*a) + np.exp(25*k)))/(2.*np.sqrt(2))

### Initialize the test function
First, let's sample the function:

In [ ]:
from fftarray import FFTArray, FFTDimension
from fftarray.constraint_solver import get_fft_dim_from_constraints

from examples.helpers import plt_fftarray, plt_complex_comparison

# initialize the backend and the coordinate grid
x_dim = get_fft_dim_from_constraints("x", pos_middle=1., pos_extent=80, n=2048, freq_middle=0.)

# get FFTArray from FFTDimension in position space
x=x_dim.fft_array(space="pos")
# numerically sampled test function in position space
f1 = f1_x(a=1.25, x=x)
plt_fftarray(f1)

### Comparison: analytic vs numeric
Now we compare the analytical with the numerical derivative that was obtained via the FFT:

In [ ]:
def derivative_pos(dim: FFTDimension, fun: FFTArray, order: int) -> FFTArray:
    """Takes the derivative of order `order along Dimension `dim` of the
    FFTArray `fun` in position space.
    """
    kernel = (1.j*2*np.pi*dim.fft_array(space="freq"))**order
    return (kernel*fun.into(space="freq")).into(space="pos")

# first order derivative
d1_analytic = f1_x_d1(a=1.25, x=x) # analytical solution
d1_numeric = derivative_pos(x_dim, f1, 1) # numerical result
plt_complex_comparison("First Order Derivative", d1_analytic, "analytic", d1_numeric, "numeric")
np.testing.assert_array_almost_equal(np.array(d1_numeric.into(space="pos")), np.array(d1_analytic.into(space="pos")), decimal=11)

# second order derivative
d2_analytic = f1_x_d2(a=1.25, x=x) # analytical solution
d2_numeric = derivative_pos(x_dim, f1, 2) # numerical result
plt_complex_comparison("Second Order Derivative", d2_analytic, "analytic", d2_numeric, "numeric")
np.testing.assert_array_almost_equal(np.array(d2_analytic.into(space="pos")), np.array(d2_numeric.into(space="pos")), decimal=9)